## Create code that processes model output into nice visualizations:

1) The files with daily cases, hospital, deaths counts across all population per districts with format output_workplaceBubblesSophie_ need to be averaged for daily trajectories
2) The files with daily cases, deaths, and in occupation by occupation (occ4) with format output_workplaceBubblesSophie_1_Economic_Status_Covid need to be averaged for daily trajectories
3) The files with daily demographics of cases and deaths by sex and gender

In [148]:
# generic packages
import os
from os.path import isfile, join
import glob
import subprocess
import re

#dataframe packages
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# plot packages
import matplotlib as mpl
import matplotlib.pyplot as plt
import pylab as plt
import seaborn as sns
sns.set(style="darkgrid")

#maps
import geopandas as gpd
import plotly.express as px
import json


In [149]:

dist_input_path= "/Users/sophieayling/Library/CloudStorage/GoogleDrive-sophie2ayling@gmail.com/My Drive/PhD/06_Data and Modelling/thesis_data/model_output/b_district_daily_counts/"
dist_output_path= "/Users/sophieayling/Library/CloudStorage/GoogleDrive-sophie2ayling@gmail.com/My Drive/PhD/06_Data and Modelling/thesis_data/model_output/b_district_daily_counts/plots/"

In [150]:
# # decide which version I am creating graphics for 

# # Define the folder path and file prefix
folder_path = dist_input_path


file_prefix = 'output_workplaceBubblesSophie_'
id_prefix='bubblesNorm'

# file_prefix = 'output_perfectMixingSophie_'
# id_prefix="perfMix"

file_prefix ='output_schoolToHomeSophie_'
id_prefix = "schoolToHome"

file_prefix ='output_schoolToComSophie_'
id_prefix = "schoolToCom"

file_prefix ='output_comWorkToHomeSophie_'
id_prefix = "comWorkersToHome"

file_prefix = 'output_workToHomeSophie_'
id_prefix = 'workToHome'

file_prefix = 'output_allToHomeSophie_'
id_prefix = 'allToHome'

# # # ## mobility scenarios set 

file_prefix = 'output_BubblesLd_'
id_prefix = 'bubblesLd'

# file_prefix = 'output_BubblesLdv2_'
# id_prefix = 'bubblesLdv2'

file_prefix = 'output_BubblesLd1a_'
id_prefix = 'bubblesLd_1a'

file_prefix = 'output_BubblesLd1b_'
id_prefix = 'bubblesLd_1b'

file_prefix = 'output_BubblesLd2a_'
id_prefix = 'bubblesLd_2a'

file_prefix = 'output_BubblesLd2b_'
id_prefix = 'bubblesLd_2b'

file_prefix = 'output_BubblesLd3a_'
id_prefix = 'bubblesLd_3a'

file_prefix = 'output_BubblesLd3b_'
id_prefix = 'bubblesLd_3b'

## 3. Cases per district - maps

In [151]:


# Use glob to find all files with the specified prefix
file_pattern = f"{folder_path}/{file_prefix}*.txt"
file_list = glob.glob(file_pattern)


# Initialize an empty list to store individual DataFrames
df_list = []

In [152]:


# Loop through the list of files and read each one into a DataFrame
for file in file_list:
    df = pd.read_csv(file, delimiter='\t')# Adjust delimiter as per your file format
    # Extract the run number from the filename
    run_number = os.path.basename(file).split('_')[2] # 2 for workplacebubbles
    df['run']=int(run_number)
    df_list.append(df)

# Concatenate all DataFrames in the list into a single DataFrame
final_df = pd.concat(df_list, ignore_index=True)
# Convert the 'run_number' column to numeric (int64)
final_df['run'] = pd.to_numeric(final_df['run'])
# Display the resulting DataFrame
final_df.head()
#final_df.metric.value_counts()

,day,metric,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,...,d_52,d_53,d_54,d_55,d_56,d_57,d_58,d_59,d_60,run
0,0,total_cases,7,6,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7
1,0,total_asympt_cases,1,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7
2,0,total_mild_cases,4,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7
3,0,total_severe_cases,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7
4,0,total_critical_cases,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7


In [153]:
# keep one with the total cases by district
tot_cases_df = final_df[final_df['metric']=='total_cases'] 

# create variable across district columns 

tot_cases_df=tot_cases_df.sort_values(by=['day', 'run'])
tot_cases_df.head()

,day,metric,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,...,d_52,d_53,d_54,d_55,d_56,d_57,d_58,d_59,d_60,run
6300,0,total_cases,11,6,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5400,0,total_cases,9,6,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
900,0,total_cases,9,7,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,3
2700,0,total_cases,6,7,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
3600,0,total_cases,13,9,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,5


In [154]:
# now reshape the data across
df_melted = pd.melt(tot_cases_df, id_vars=['day', 'run'], value_vars=[f'd_{i}' for i in range(1, 61)], var_name='district', value_name='tot_cases')

# create version which is without the d_
df_melted['dist_no']= df_melted['district'].str.replace('d_', '')

df_melted.head()


,day,run,district,tot_cases,dist_no
0,0,1,d_1,11,1
1,0,2,d_1,9,1
2,0,3,d_1,9,1
3,0,4,d_1,6,1
4,0,5,d_1,13,1


In [155]:

#group the data by daily district cases (aggregating runs) but keeping districts independent
r_data = df_melted.groupby(['day','dist_no'])['tot_cases'].mean().reset_index()
# Round the 'value' column to 1 decimal place
r_data['tot_cases'] = r_data['tot_cases'].round(1)
## making the district numbers numeric 

# Generate the logarithm of total cases
# To avoid issues with log(0), ensure no zero values in 'tot_cases' before applying log
r_data['log_tot_cases'] = np.log(r_data['tot_cases'].replace(0, np.nan))


r_data['dist_no']=pd.to_numeric(r_data['dist_no'])
r_data=r_data.sort_values(by=['day', 'dist_no'])

r_data.to_csv (dist_output_path+f'{id_prefix}_agg_case_counts_dist.csv')
r_data.sort_values(by='dist_no')
r_data.head()
r_data['log_tot_cases'].max()
r_data.head()

,day,dist_no,tot_cases,log_tot_cases
0,0,1,8.6,2.151762
11,0,2,7.9,2.066863
22,0,3,0.0,NaN
33,0,4,0.0,NaN
44,0,5,0.0,NaN


In [156]:
# extract statistics on districts with top peak and highest cumulative deaths

In [157]:
peak_per_dist = r_data.groupby('dist_no', as_index=False)['tot_cases'].max()

r_data_t5_peak = peak_per_dist.nlargest(5,'tot_cases')
# r_data_t5_death = r_data.nlargest(5,'metric_died_count')

r_data_t5_peak.head()
r_data_t5_peak.to_csv(dist_output_path+f'_top5_peak_dists_{id_prefix}.csv')

In [158]:
stop

NameError: name 'stop' is not defined

## Do subset of data which is just the high leavers 

In [ ]:
r_data_5 = r_data[r_data['dist_no'].isin([23,31,45,18,21])]
r_data_12 = r_data[r_data['dist_no'].isin([23,31,45,18,21,52,34,17,53,25,16,19])]

In [ ]:
r_data_5.head()

In [ ]:
# now graph

# ensure I have enough colours in the palette 
palette = sns.color_palette("husl", 60)

# Plotting the data
plt.figure(figsize=(10, 5))
sns.lineplot(data=r_data, x="day", y="log_tot_cases", hue="dist_no", palette=palette) #, style="", , err_style="band"


# Adding titles and labels
plt.title(f'{id_prefix}_Cases Over Time by District', size=18)
plt.xlabel('Day')
plt.ylabel('Total Number of Cases')
plt.legend(title='District', loc= 'upper left', bbox_to_anchor=(1,1), ncol=3, prop={'size': 8})
plt.xlim(0,100)
plt.ylim(0,10)
plt.grid(True)
plt.subplots_adjust(right=0.7)  # Adjust the right margin to make room for the legend

# export the plot 
plt.savefig(dist_output_path+f'{id_prefix}cases_over_time_district.png', dpi=300)


In [ ]:
# now graphn without legend

# ensure I have enough colours in the palette 
palette = sns.color_palette("husl", 60)

# Plotting the data
plt.figure(figsize=(10, 5))
sns.lineplot(data=r_data, x="day", y="tot_cases", hue="dist_no", palette=palette, legend=False) #, style="", , err_style="band"


# Adding titles and labels
plt.title(f'{id_prefix}_Cases Over Time by District', size=18)
plt.xlabel('Day')
plt.ylabel('Total Number of Cases')
#plt.legend(title='District', loc= 'upper left', bbox_to_anchor=(1,1), ncol=3, prop={'size': 8})
plt.xlim(0,100)
plt.ylim(0,7000)
plt.grid(True)
plt.subplots_adjust(right=0.7)  # Adjust the right margin to make room for the legend

# export the plot 
plt.savefig(dist_output_path+f'{id_prefix}cases_over_time_district_nolegend.png', dpi=300, facecolor="white")

r_data.head()

In [ ]:
# now plot - districts with the highest case numbers overall

map_input_path= "/Users/sophieayling/Library/CloudStorage/GoogleDrive-sophie2ayling@gmail.com/My Drive/PhD/06_Data and Modelling/thesis_data/shapefiles/60_districts/"
dist_shape="ZWE_adm2.shp"

dists=gpd.read_file(map_input_path+dist_shape)

dists.plot()
plt.show()
dists.head()
dists['dist_no']= dists['ID_2']

In [ ]:
#group the data by daily district cases (aggregating runs) but keeping districts independent.
r_data_tot = r_data.groupby('dist_no')['tot_cases'].sum().reset_index()
# Round the 'value' column to 1 decimal place
r_data_tot['tot_cases'] = r_data_tot['tot_cases'].round(1)
r_data_tot.head()

In [ ]:
merged_gdf=dists.merge(r_data_tot, on= 'dist_no')

# Rename the column 'NAME_2' to 'name'
merged_gdf.rename(columns={'NAME_2': 'name'}, inplace=True)

# Define color map
cmap = 'magma_r' # if you add _r at the end it inverts the color map

# other options include viridis, plasma, inferno, magma, cividis

# Create plot
fig, ax = plt.subplots(1, 1, figsize=(10, 8))

# Plot the GeoDataFrame with the specified column and color map
merged_gdf.plot(column='tot_cases', cmap=cmap, linewidth=0.8, ax=ax, edgecolor='0.8', legend=True,vmin=0, vmax=80000 )

# Add title
plt.title(f'{id_prefix}_Total Cases by District', fontsize=18)


# export
plt.savefig(dist_output_path+f'{id_prefix}dist_tot_cases.png', dpi=300)
plt.show()


## add a simple bar graph for total cases

In [ ]:
merged_gdf.head()
bar_data = merged_gdf[['name', 'dist_no', 'tot_cases']]
bar_data = bar_data.sort_values(by='tot_cases', ascending=False)
bar_data.info()
plt.figure(figsize=(10,14))
plt.tight_layout()
# Normalize the ranks to create a gradient effect
norm = np.linspace(0, 1, len(bar_data))
# Create a color gradient from red to blue
colors = sns.color_palette("RdBu", as_cmap=True)(norm)
sns.barplot(data=bar_data, x="tot_cases", y="name", palette=colors)
plt.ylabel("District")
plt.xlabel("Total cases")
plt.xlim(0, 85000)
plt.title(f'{id_prefix}: Total cases over 100 days', fontsize=18 )
# export
plt.savefig(dist_output_path+f'{id_prefix}graph_dist_tot_cases.png', dpi=300)

In [ ]:
# use plotly to make an interactive map instead

import plotly.express as px


# Convert the GeoDataFrame back to a geographic CRS (EPSG:4326) for Plotly
gdf= merged_gdf.to_crs(epsg=4326)

# covert geodataframe to geojson
geojson_data= json.loads(gdf.to_json())

# Coordinates for Zimbabwe's approximate center
zimbabwe_center = {"lat": -19.015438, "lon": 29.154857}
    
# Create the choropleth map using Plotly
fig = px.choropleth_mapbox(
    gdf,
    geojson=geojson_data,
    locations='dist_no',  # Column in merged_gdf to match with the GeoJSON
    featureidkey="properties.dist_no",  # Property in GeoJSON to match locations - if you don't do this it gets super confused, this command is essential
    color='tot_cases',  # Column to use for color
    color_continuous_scale="magma_r",
    center=zimbabwe_center,
    mapbox_style="carto-positron",
    zoom=5,
    title='Total Cases by District', 
    hover_data={'dist_no': True, 'name': True, 'tot_cases': True}  # Add district name and other relevant data to hover info

)

# Update the layout to set the color scale and other properties
fig.update_layout(
    coloraxis_colorbar=dict(
        title="Total Cases",
        ticks="outside"
    ),
    margin={"r":0,"t":0,"l":0,"b":0}
)

# Show the map
fig.show()





In [ ]:
gdf.head(60)